This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [ ]:
#!micromamba install pandera pandera-hypotheses pandera-io pandera-geopandas -c conda-forge -y

In [51]:
%load_ext autoreload
%autoreload 2

In [52]:
from pathlib import Path
import sys
import logging

from IPython.lib import backgroundjobs as bg

scripts_dir = Path('..').joinpath('src')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from pipelines.pipes import get_pipes, execution_order, filter_pipes

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)
jobs = bg.BackgroundJobManager()

In [ ]:
# def execute_pipes(pipes):
#     ordered_pipes = execution_order(pipes)
#     for pipe in ordered_pipes:
#         jobs.new(pipe().execute)

# # this code will execute all pipelines selected in background.

# mypipes_subset = filter_pipes(get_pipes(), ['eez_tiles'])
# execute_pipes(mypipes_subset)

In [2]:
get_pipes()

{'eez_intermediate': pipelines.intermediate_pipes.EEZIntermediatePipe.EEZIntermediatePipe,
 'mpaatlas_intermediate': pipelines.intermediate_pipes.MpaAtlasIntermediatePipe.MpaAtlasIntermediatePipe,
 'mpa_intermediate': pipelines.intermediate_pipes.MpasIntermediatePipe.MpasIntermediatePipe,
 'protectedseas_intermediate': pipelines.intermediate_pipes.ProtectedSeasIntermediatePipe.ProtectedSeasIntermediatePipe,
 'eez_tiles': pipelines.tiles_pipes.EEZTilesPipe.EEZTilesPipe,
 'mpaatlas_tiles': pipelines.tiles_pipes.MpaAtlasTilesPipe.MpaAtlasTilesPipe,
 'mpa_tiles': pipelines.tiles_pipes.MpasTilesPipe.MpasTilesPipe,
 'protectedseas_tiles': pipelines.tiles_pipes.ProtectedSeasTilesPipe.ProtectedSeasTilesPipe,
 'regions_tiles': pipelines.tiles_pipes.RegionsTilesPipe.RegionsTilesPipe,
 'habitats_precalc': pipelines.precalc_pipes.HabitatsPipe.HabitatsStatsPipe,
 'lfp_precalc': pipelines.precalc_pipes.LFPStatsPipe.LFPStatsPipe,
 'eez_locations_precalc': pipelines.precalc_pipes.LocationsPipe.Locatio

In [13]:
mypipes_subset = filter_pipes(get_pipes(), ["mpaatlas_intermediate"])

In [14]:
for n, pipe in mypipes_subset.items():
    new_pipe = pipe()
    new_pipe.extract().transform()

DEBUG:pipelines.settings:/home/mambauser/data
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate running at 2024-03-13 08:26:49.462776: starting extract...
INFO:helpers.utils:File /home/mambauser/data/mpaatlas_intermediate/mpatlas_assess_zone.geojson already exists.
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate finish at 2024-03-13 08:26:49.463871: Success executing extract
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate running at 2024-03-13 08:26:49.464149: starting transform...
/home/mambauser/src/pipelines/intermediate_pipes/MpaAtlasIntermediatePipe.py:84: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate finish at 2024-03-13 08:27:17.038267: Success executing transform


In [53]:
from pathlib import Path
from typing import Union
import pandas as pd
import geopandas as gpd
import numpy as np

from helpers.strapi import Strapi

from data_commons.loader import load_regions

from pipelines.output_schemas import (
    StablishmentStageSchema,
    FPLSchema,
    ProtectionLevelSchema,
    MPAsSchema,
    MPAsTableStatsSchema,
    HabitatsSchema,
)

In [7]:
eez_folder = Path("../data/eez_intermediate").absolute()
mpa_folder = Path("../data/mpa_intermediate").absolute()
mpaatlas_folder = Path("../data/mpaatlas_intermediate").absolute()
protectedseas_folder = Path("../data/protectedseas_intermediate").absolute()
habitats_folder = Path("../data/habitats_intermediate").absolute()


location_code = pd.read_csv(eez_folder.joinpath("locations_code.csv"))

In [47]:
habitats_intermediate = pd.read_csv(habitats_folder.joinpath("habitats4.csv"), keep_default_na=False)

In [8]:

mpa_intermediate = gpd.read_file(mpa_folder.joinpath("mpa_intermediate", "mpa_intermediate.shp"))

In [10]:
protectedseas_intermediate = gpd.read_file(
    protectedseas_folder.joinpath("protectedseas_intermediate", "protectedseas_intermediate.shp")
)

In [15]:
mpaatlas_intermediate = gpd.read_file(
    mpaatlas_folder.joinpath("mpaatlas_intermediate", "mpaatlas_intermediate.shp").as_posix()
)

In [16]:
mpaatlas_intermediate[mpaatlas_intermediate["location_i"]=="PAN"]

,wdpa_id,name,designatio,location_i,establishm,protection,protecti_1,year,geometry
285,555705293,Cordillera de Coiba - Zona de Pesca Comercial ...,None,PAN,actively managed,high,fully or highly protected,2021,"POLYGON ((-83.22383 5.87577, -83.22383 6.50159..."
367,555705293,Cordillera de Coiba - Zona de Reserva Marina,Marine Protected Area,PAN,actively managed,full,fully or highly protected,2021,"POLYGON ((-83.61346 6.50156, -83.22383 6.50159..."


In [17]:
def separate_parent_iso(df: pd.DataFrame, iso_column="location_i", separator=";") -> pd.DataFrame:
    df[iso_column] = (
        df[iso_column].str.replace(" ", "").str.replace(":", separator).str.split(separator)
    )
    return df.explode(iso_column)


def calculate_area(df: pd.DataFrame, output_area_column="area_km2") -> pd.DataFrame:
    df[output_area_column] = (df.to_crs("ESRI:54009")["geometry"].area / 10**6).round(2)
    return df


def calculate_global_area(
    df: pd.DataFrame, gby_col: list, output_area_column="area_km2", iso_column="location_i"
) -> pd.DataFrame:
    global_area = (
        df.groupby([*gby_col])
        .agg({output_area_column: "sum"})
        .reset_index()
        .assign(**{iso_column: "GLOB"})
    )
    return pd.concat([global_area, df], ignore_index=True)


def add_region_iso(df: pd.DataFrame, iso_column) -> pd.DataFrame:
    regions = load_regions()

    def find_region_iso(iso: str) -> Union[str, None]:
        filtered_regions = list(filter(lambda x: iso in x["country_iso_3s"], regions.get("data")))
        return filtered_regions[0]["region_iso"] if len(filtered_regions) > 0 else None

    return df.assign(region=lambda row: row[iso_column].apply(find_region_iso))


def mpaatlas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "location_i"})
    )

    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "location_i"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def protectedseas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "iso"})
    )
    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "iso"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def batch_export(df: pd.DataFrame, batch_size: int, schema: object, folder: Path, filename: str):
    prev = 0
    for idx, size in enumerate(range(batch_size, len(df.index) + batch_size, batch_size)):
        schema(df[(df.index > prev) & (df.index < size)]).to_csv(
            folder.joinpath(f"{filename}_{idx}.csv"),
            index=True,
            encoding="utf-8",
        )
        prev = size


def fix_monaco(df: pd.DataFrame, iso_column="location_i", area_column="area_km2") -> pd.DataFrame:
    df.loc[df[iso_column] == "MCO", area_column] = 288
    return df


def set_area(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(area_km2=df[["area_km2_y", "area_km2_x"]].max(axis=1))


def output(
    df: pd.DataFrame, iso_column: str, rep_d: dict, rename: dict, drop_cols: list
) -> pd.DataFrame:
    if iso_column:
        locations_code = pd.read_csv(
            eez_folder.joinpath("locations_code.csv"), keep_default_na=False
        )
        df = df.join(locations_code.set_index("code"), on=iso_column, how="left")
    return (
        df.replace(rep_d)
        .rename(columns=rename)
        .drop(columns=drop_cols)
        .assign(
            id=df.index + 1,
        )
        .set_index("id")
    )


def filter_location(df: pd.DataFrame) -> pd.DataFrame:
    return df[~df.location.isna()]

In [48]:
habitats_intermediate

,location_id,protected_area,total_area,habitat_name,year
0,AGO,0.0,3.395671,cold-water corals,2024
1,ALB,0.0,5.986479,cold-water corals,2024
2,ARG,6.98422602063557,61.826344,cold-water corals,2024
3,ATA,5.41915117560982,18.889068,cold-water corals,2024
4,ATG,0.0,0.997747,cold-water corals,2024
...,...,...,...,...,...
611,AS,21277.219999999998,74292.673146,mangroves,2020
612,EU,732.14375,1246.189677,mangroves,2020
613,NA,2097.74,2415.418557,mangroves,2020
614,SA,27151.739999999998,39893.444608,mangroves,2020


In [62]:
habitat_stats.replace({"protectedArea":{"": None}}).astype({"protectedArea": "float64"}).info()

<class 'pandas.core.frame.DataFrame'>
Index: 616 entries, 1 to 616
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   protectedArea  570 non-null    float64
 1   totalArea      616 non-null    float64
 2   habitat        616 non-null    int64  
 3   year           616 non-null    int64  
 4   location       616 non-null    int64  
dtypes: float64(2), int64(3)
memory usage: 28.9 KB


In [64]:
habitat_stats = habitats_intermediate.pipe(
    output,
    iso_column="location_id",
    rep_d={
        "habitat_name": {
            "saltmarshes": 1,
            "seagrasses": 2,
            "warm-water corals": 3,
            "cold-water corals": 4,
            "mangroves": 5,
            "seamounts": 6,
        },
        "protected_area": {"": 0},
    },
    rename={
        "protected_area": "protectedArea",
        "total_area": "totalArea",
        "habitat_name": "habitat",
    },
    drop_cols=["location_id"],
)
HabitatsSchema(habitat_stats).to_csv(habitats_folder.joinpath("habitats_stats.csv"), index=True)

In [18]:
mpa_atlas_table = (
    mpaatlas_intermediate.pipe(calculate_area)
)

In [19]:
test = (
    mpa_atlas_table.pipe(calculate_global_area, gby_col=["establishm"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["establishm"])
    .pipe(fix_monaco, iso_column="location_i", area_column="area_km2")
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "establishm": {
                "actively managed": 4,
                "implemented": 6,
                "designated": 5,
                "proposed or committed": 3,
                "unknown": 1,
            }
        },
        rename={"establishm": "mpaa_establishment_stage", "area_km2": "area"},
        drop_cols=["location_i"],
    )
).assign(year=2023, protection_status=1)

StablishmentStageSchema(test[~test.location.isna()]).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_stablishment.csv"), index=True
)

In [20]:
test2 = (
    mpa_atlas_table.pipe(calculate_global_area, gby_col=["protecti_1"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["protecti_1"])
    .pipe(fix_monaco, iso_column="location_i", area_column="area_km2")
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "protecti_1": {
                "fully or highly protected": 1,
                "less protected or unknown": 2,
            }
        },
        rename={"protecti_1": "mpaa_protection_level", "area_km2": "area"},
        drop_cols=[],
    )
)
ProtectionLevelSchema(test2[~test2.location.isna()].assign(year=2023)).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_protection_level.csv"), index=True
)

In [21]:
protected_seas_table = (
    protectedseas_intermediate.pipe(calculate_area)
)

In [22]:
test3 = (
    protected_seas_table.pipe(calculate_global_area, gby_col=["FPS_cat"], iso_column="iso")
    .pipe(separate_parent_iso, iso_column="iso")
    .replace(
        {
            "iso": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="iso")
    .pipe(protectedseas_calculation, gby_col=["FPS_cat"])
    .pipe(fix_monaco, iso_column="iso", area_column="area_km2")
    .pipe(
        output,
        iso_column="iso",
        rep_d={
            "FPS_cat": {
                "highly": 1,
                "moderately": 2,
                "less": 3,
            }
        },
        rename={"FPS_cat": "fishing_protection_level", "area_km2": "area"},
        drop_cols=["iso"],
    )
)

FPLSchema(test3).to_csv(protectedseas_folder.joinpath("lfp.csv"), index=True)

In [23]:
mpa_intermediate

,WDPAID,WDPA_PID,PA_DEF,NAME,GIS_M_AREA,STATUS,STATUS_YR,PARENT_ISO,geometry
0,1,1,1,Diamond Reef and Salt Fish Tail Reef,14.636135,Designated,1973,ATG,"POLYGON ((-61.82494 17.18497, -61.82497 17.184..."
1,2,2,1,Palaster Reef,3.845623,Designated,1973,ATG,"POLYGON ((-61.74007 17.52001, -61.77174 17.526..."
2,27,27,1,Folkstone,9.989930,Designated,1980,BRB,"POLYGON ((-59.63212 13.17370, -59.63263 13.168..."
3,46,46,1,Reserva Biológica Atol Das Rocas,353.837622,Designated,1979,BRA,"POLYGON ((-33.64137 -3.79099, -33.64151 -3.791..."
4,57,57,1,Parque Nacional Do Cabo Orange,2270.594697,Designated,1980,BRA,"POLYGON ((-50.85381 2.81364, -50.85443 2.81359..."
...,...,...,...,...,...,...,...,...,...
18727,555594046,555594046,1,Mashtan Island,NaN,Designated,2002,BHR,"POLYGON ((4759827.90237 3149355.27177, 4759827..."
18728,313506,313506,1,Arad Bay,NaN,Designated,2003,BHR,"POLYGON ((4743197.11699 3203279.17554, 4743197..."
18729,145813,145813,1,Hawar Island and the Surrounding Terretorial Sea,NaN,Designated,1996,BHR,"POLYGON ((4771270.71072 3129994.93264, 4771269..."
18730,145812,145812,1,Tubli Bay,NaN,Designated,2006,BHR,"POLYGON ((4738737.37964 3195543.87956, 4738737..."


In [26]:
test4[test4.area.isna()]

,wdpaid,protection_status,name,area,mpaa_establishment_stage,year,location
id,,,,,,,


In [25]:
test4 = (
    mpa_intermediate.fillna(0)
    .replace(
        {
            "PARENT_ISO": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(
        output,
        iso_column="PARENT_ISO",
        rep_d={
            "STATUS": {
                "Adopted": 4,
                "implemented": 6,
                "Established": 6,
                "Designated": 5,
                "Proposed": 3,
                "Inscribed": 3,
                "unknown": 1,
            },
            "PA_DEF": {"0": 2, "1": 1},
            "STATUS_YR": {0: pd.NA},
        },
        rename={
            "PARENT_ISO": "iso",
            "PA_DEF": "protection_status",
            "GIS_M_AREA": "area",
            "STATUS_YR": "year",
            "WDPA_PID": "wdpaid",
            "NAME": "name",
            "STATUS": "mpaa_establishment_stage",
        },
        drop_cols=["geometry", "WDPAID", "iso"],
    )
    .astype({"year": "Int64"})
)

prev = 0
for idx, size in enumerate(range(5000, len(test4.index) + 5000, 5000)):
    MPAsSchema(test4[(test4.index > prev) & (test4.index < size)]).to_csv(
        mpa_folder.joinpath(f"mpa_{idx}.csv"),
        index=True,
        encoding="utf-8",
    )
    prev = size

In [27]:
mpa_atlas_merge = mpa_atlas_table.pipe(separate_parent_iso, iso_column="location_i").replace(
    {
        "location_i": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

protectedseas_merge = protected_seas_table.pipe(separate_parent_iso, iso_column="iso").replace(
    {
        "iso": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

In [129]:
Final = (
    test4.assign(mpa=test4.index)
    .merge(
        mpa_atlas_merge[["establishm", "wdpa_id", "protecti_1", "area_km2"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .merge(
        protectedseas_merge[["site_id", "wdpa_id", "area_km2", "FPS_cat"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .pipe(set_area)
    .pipe(filter_location)
    .reset_index(drop=True)
    )
Final_output = Final.pipe(
        output,
        iso_column=None,
        rep_d={
            "protecti_1": {
                "fully or highly protected": 1,
                "less protected or unknown": 2,
            },
            "FPS_cat": {
                "highly": 1,
                "moderately": 2,
                "less": 3,
            },
            "establishm": {
                "actively managed": 4,
                "implemented": 6,
                "designated": 5,
                "proposed or committed": 3,
                "unknown": 1,
            },
        },
        rename={
            "establishm": "mpaa_establishment_stage",
            "protecti_1": "mpaa_protection_level",
            "FPS_cat": "fishing_protection_level",
        },
        drop_cols=[
            "wdpaid",
            "wdpa_id_x",
            "wdpa_id_y",
            "area_km2",
            "area_km2_x",
            "area_km2_y",
            "mpaa_establishment_stage",
            "protection_status",
            "name",
            "site_id",
        ],
    )

batch_export(Final_output, 5000, MPAsTableStatsSchema, mpa_folder, "mpa_join_mpatlas_prot")

In [132]:
Final.head(10)

,wdpaid,protection_status,name,area,mpaa_establishment_stage,year,location,mpa,establishm,wdpa_id_x,protecti_1,area_km2_x,site_id,wdpa_id_y,area_km2_y,FPS_cat,area_km2
0,1,1,Diamond Reef and Salt Fish Tail Reef,14.636135,5,1973,15.0,1,NaN,NaN,NaN,NaN,AIAG5,1,14.67,highly,14.67
1,2,1,Palaster Reef,3.845623,5,1973,15.0,2,NaN,NaN,NaN,NaN,AIAG14,2,3.23,less,3.23
2,27,1,Folkstone,9.989930,5,1980,27.0,3,NaN,NaN,NaN,NaN,AIBB3,27,11.11,highly,11.11
3,46,1,Reserva Biológica Atol Das Rocas,353.837622,5,1979,26.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57,1,Parque Nacional Do Cabo Orange,2270.594697,5,1980,26.0,5,NaN,NaN,NaN,NaN,AIBRA459,57,6616.77,highly,6616.77
5,78,1,RMS Rhone,2.603766,5,1980,59.0,6,NaN,NaN,NaN,NaN,AIBVI9,78,2.90,highly,2.90
6,80,1,West Dog,0.089941,5,1974,59.0,7,NaN,NaN,NaN,NaN,AIBVI8,80,0.08,highly,0.08
7,81,1,Devils Bay,0.031665,5,1964,59.0,8,NaN,NaN,NaN,NaN,AIBVI3,81,0.19,highly,0.19
8,92,1,Cabo de Hornos,175.910035,5,1945,30.0,9,NaN,NaN,NaN,NaN,AICHL022,92,588.07,moderately,588.07
9,97,1,Archipielago Juan Fernadez,55.976602,5,1935,30.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
protectedseas_merge.wdpa_id.unique().shape

(10786,)

In [30]:
protectedseas_merge.wdpa_id.str.replace(' ','').str.split(';').explode().unique().shape

(11984,)

In [36]:
test4[test4.area == 0]

,wdpaid,protection_status,name,area,mpaa_establishment_stage,year,location
id,,,,,,,
1269,41057_A,1,Shell Beach Protected Area,0.0,5,2011,70.0
1424,65371_A,1,Rathtrevor Beach Park,0.0,5,1967,29.0
1554,67785_B,1,Hunter Estuary Wetlands,0.0,5,2002,16.0
3792,167356_A,1,Homathko Estuary Park,0.0,5,1997,29.0
4834,302326_A,1,Polletts Cove - Aspy Fault Wilderness Area,0.0,5,1999,29.0
...,...,...,...,...,...,...,...
18728,555594046,1,Mashtan Island,0.0,5,2002,22.0
18729,313506,1,Arad Bay,0.0,5,2003,22.0
18730,145813,1,Hawar Island and the Surrounding Terretorial Sea,0.0,5,1996,22.0


In [133]:
Final[Final.area == 0]

,wdpaid,protection_status,name,area,mpaa_establishment_stage,year,location,mpa,establishm,wdpa_id_x,protecti_1,area_km2_x,site_id,wdpa_id_y,area_km2_y,FPS_cat,area_km2
1314,41057_A,1,Shell Beach Protected Area,0.0,5,2011,70.0,1269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1471,65371_A,1,Rathtrevor Beach Park,0.0,5,1967,29.0,1424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1601,67785_B,1,Hunter Estuary Wetlands,0.0,5,2002,16.0,1554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3843,167356_A,1,Homathko Estuary Park,0.0,5,1997,29.0,3792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4892,302326_A,1,Polletts Cove - Aspy Fault Wilderness Area,0.0,5,1999,29.0,4834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18904,555594046,1,Mashtan Island,0.0,5,2002,22.0,18728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18905,313506,1,Arad Bay,0.0,5,2003,22.0,18729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18906,145813,1,Hawar Island and the Surrounding Terretorial Sea,0.0,5,1996,22.0,18730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18907,145812,1,Tubli Bay,0.0,5,2006,22.0,18731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
test_Final = Final[~Final.area.isna()]# [
    # ["wdpaid", "name", "area_km2_x", "area", "area_km2_y", "FPS_cat", "area_km2", "protecti_1"]
# ]
test_Final

,area,year,location,mpa,mpaa_protection_level,fishing_protection_level
id,,,,,,
1,14.636135,1973,15.0,1,NaN,1.0
2,3.845623,1973,15.0,2,NaN,3.0
3,9.989930,1980,27.0,3,NaN,1.0
4,353.837622,1979,26.0,4,NaN,NaN
5,2270.594697,1980,26.0,5,NaN,1.0
...,...,...,...,...,...,...
18905,0.000000,2002,22.0,18728,NaN,NaN
18906,0.000000,2003,22.0,18729,NaN,NaN
18907,0.000000,1996,22.0,18730,NaN,NaN


In [ ]:
# test_Final["area"] = test_Final.area_km2_y - test_Final.area_km2_x
# test_Final[(test_Final.area_km2_1 != 0) & (test_Final.area_km2_1.notna())]

In [126]:
base_url = "https://30x30-dev.skytruth.org/cms"

strapi = Strapi(url=base_url)
strapi.login(
)

In [124]:
strapi.getCollectionMetadata("mpa-protection-coverage-stat")

[{'uid': 'api::mpa-protection-coverage-stat.mpa-protection-coverage-stat',
  'isDisplayed': True,
  'apiID': 'mpa-protection-coverage-stat',
  'kind': 'collectionType',
  'info': {'singularName': 'mpa-protection-coverage-stat',
   'pluralName': 'mpa-protection-coverage-stats',
   'displayName': 'MPA Protection Coverage Stats',
   'description': ''},
  'options': {'draftAndPublish': False},
  'pluginOptions': {},
  'attributes': {'id': {'type': 'integer'},
   'mpa': {'type': 'relation',
    'relation': 'manyToOne',
    'target': 'api::mpa.mpa',
    'inversedBy': 'mpa_protection_coverage_stats',
    'targetModel': 'api::mpa.mpa',
    'relationType': 'manyToOne'},
   'fishing_protection_level': {'type': 'relation',
    'relation': 'oneToOne',
    'target': 'api::fishing-protection-level.fishing-protection-level',
    'targetModel': 'api::fishing-protection-level.fishing-protection-level',
    'relationType': 'oneToOne'},
   'mpaa_protection_level': {'type': 'relation',
    'relation': 'on

In [89]:
# strapi.deleteCollectionData("mpa-protection-coverage-stat", list(range(1, 18914)))

In [120]:
#strapi.getCollectionData("mpa-protection-coverage-stats")

In [121]:
strapi.session.get(f"{base_url}/api/mpa-protection-coverage-stats")

<Response [200]>

In [127]:
strapi.importCollectionData(
    "mpa-protection-coverage-stat", Path("/home/mambauser/data/mpa_intermediate/mpa_join_mpatlas_prot_2.csv")
)

HTTPError: 401 Client Error: Unauthorized for url: https://30x30-dev.skytruth.org/cms/api/import-export-entries/content/import